<a href="https://colab.research.google.com/github/shubhamsrivastava951/NLP/blob/main/NER_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

 #download data
id = '1YnpJzqcTAwikoAzKuqKtkiXMjCphud51'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train.csv')

id = '1NElN4SlJoimeDbV8MoV8LsBmNYAOVcDw'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('val.csv')

id = '1E2XvfBpX8OUKPdkFtmepaRguXXdttbhX'
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('test_without_labels.csv')

import pandas as pd
import numpy as np
import datetime

import sys
import nltk
import re
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

import sys

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# Data PreProcessing

In [ ]:
#read csv fiels into pandas dataframe
df_train= pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")
df_test = pd.read_csv("test_without_labels.csv")

In [ ]:
df_train.info()
# Check for null/missing values
print(f"\nNull values for training data set: {df_train.isnull().values.sum()}")
print(f"\nNull values for test data set: {df_val.isnull().values.sum()}")
print(f"\nNull values for val data set: {df_test.isnull().values.sum()}")

#check for any NAN values
print(f"\nAny NAN values in Train: {df_train.isnull().values.any()}")
print(f"\nAny NAN values in Val: {df_val.isnull().values.any()}")
print(f"\nAny NAN values in test: {df_val.isnull().values.any()}")

#Shape of dataframe
print(f"\n Train shape: {df_train.shape}")
print(f"\n Val shape: {df_val.shape}")
df_train.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sents   573 non-null    object
 1   labels  573 non-null    object
dtypes: object(2)
memory usage: 9.1+ KB

Null values for training data set: 0

Null values for test data set: 0

Null values for val data set: 0

Any NAN values in Train: False

Any NAN values in Val: False

Any NAN values in test: False

 Train shape: (573, 2)

 Val shape: (191, 2)


,sents,labels
0,Operation Steel Curtain ( Arabic : ا ل ح ج ا ب...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,The hospital has facilities for MRI and CT sca...,B-Location I-Location O O O O O O O O O O O O ...
2,The operation was important in that it was the...,O O O O O O O O O O O O O O B-Organisation I-O...
3,This was my first visit to Uzbekistan and an i...,O O B-Person O O O B-Location O O O O O O O O ...
4,The group was founded by Sheikh Abu Omar al - ...,B-Organisation I-Organisation O O O B-Person I...
5,"And hopefully , by reading , we can achieve th...",O O O O O O O O O O O O O O
6,But nobody can leave now .,O O O O O O
7,The Members of the Security Council are encour...,B-Organisation I-Organisation I-Organisation I...
8,Iraq,O
9,We extend our deepest condolences to the famil...,B-Organisation O O O O O B-Organisation I-Orga...


In [ ]:
#Validation set
df_val.head(5)

,sents,labels
0,For 14 - year - old Amjad it is safer there th...,O B-Quantity I-Quantity I-Quantity I-Quantity ...
1,". Near Al Huwayjah , one strike engaged an ISI...",O O B-Location I-Location O B-Quantity B-Weapo...
2,Philip Hammond condemns apparent barrel bomb a...,B-Person I-Person O O B-Weapon I-Weapon O O B-...
3,Foreign Secretary Boris Johnson welcomes the U...,B-Person I-Person I-Person I-Person O O B-Docu...
4,Sadiq stood behind Karim when the musician pla...,B-Person O O B-Person O B-Person I-Person O O ...


In [ ]:
#convert all sentences to lower in train,val and test set
df_train['sents']=df_train['sents'].str.lower()
df_val['sents']=df_val['sents'].str.lower()
df_test['sents']=df_test['sents'].str.lower()
# df_train.head(5)

In [ ]:
# def remove_extra_dots(x):
#   for sent in x:
#     return re.sub('[.](?!\d*$)', '',x)
#     # return re.sub('``',"' '",x)
#     #split "  to single quotes

# df_train['sents']=df_train['sents'].apply(lambda x:remove_extra_dots(x))

In [ ]:
#Creating list of sentences and labels
sentences = df_train['sents'].tolist()
labels= df_train['labels'].tolist()
print("Training set")
print(f"First Sentence: {sentences[0]}")
print(f"NER Tags of first Sentence: {labels[0]}")

print("\n Val set")
val_sentences = df_val['sents'].tolist()
val_labels= df_val['labels'].tolist()
print(f"First val Sentence: {val_sentences[0]}")
print(f"NER Tags of first val Sentence: {val_labels[0]}")

print("\n Test set")
test_sentences = df_test['sents'].tolist()
print(f"First Test Sentence: {test_sentences[0]}")

Training set
First Sentence: operation steel curtain ( arabic : ا ل ح ج ا ب ا ل ف و ل ا ذ ي al hejab elfulathi ) was a military operation executed by coalition forces in early november 2005 to reduce the flow of foreign insurgents crossing the border and joining the iraqi insurgency .
NER Tags of first Sentence: O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O B-Organisation I-Organisation O B-Temporal I-Temporal I-Temporal O O O O O B-Organisation I-Organisation O O O O O O B-Nationality O O

 Val set
First val Sentence: for 14 - year - old amjad it is safer there than being above ground , and over time his enthusiasm for the place has earned him the role of " " deputy librarian " " . "
NER Tags of first val Sentence: O B-Quantity I-Quantity I-Quantity I-Quantity I-Quantity B-Person O O O O O O O O O O O O B-Person O O O O O O O O O O O O O O O O O O

 Test set
First Test Sentence: carter thanked abadi for nearly two years of a close personal partnership , and noted the con

In [ ]:
#getting POS TAGS for each word in the sentences
# import spacy
# from spacy import displacy
# import en_core_web_sm

# #nlp=spacy.load('en_core_web_trf')
# nlp = en_core_web_sm.load()
# POS_TAGS=[]
# D_Tags=[]
# def get_tag(x):
#   pTags=[]
#   dTags=[]
#   for words in nlp(x):
#     pTags.append(words.tag_)
#     dTags.append(words.dep_)
#   POS_TAGS.append(' '.join(pTags))
#   D_Tags.append(' '.join(dTags))

# df_train['sents'].apply(lambda x: get_tag(x))
# print(f"First Sentence --> correspoding POS Tags: {POS_TAGS[0]}")
# print(f"\nFirst Sentences  --> correspoding Dependency Tags: {D_Tags[0]}")

## POS Tags

In [ ]:
POS_TAGS=[]
# D_Tags=[]
val_POS_TAGS=[]
test_POS_TAGS=[]

#function to get POS TAGS
def get_tag(x,POS='Train'):
  pTags=[]
  temp=[]
  #nltk POS tags function
  tags_tp_list=nltk.pos_tag(x.split())

  for words,tags in tags_tp_list:
    temp.append(tags)
  if POS=='Test':
    test_POS_TAGS.append(' '.join(temp))
  elif POS=='Val':
    val_POS_TAGS.append(' '.join(temp))
  else:
    POS_TAGS.append(' '.join(temp))


df_train['sents'].apply(lambda x: get_tag(x))
df_val['sents'].apply(lambda x: get_tag(x,POS='Val'))
df_test['sents'].apply(lambda x: get_tag(x,POS='Test'))
print(f"(First Sentence --> correspoding POS Tags: {POS_TAGS[0]})")

(First Sentence --> correspoding POS Tags: NN NN NN ( JJ : JJ NNP NNP NNP NNP NNP NNP NNP NNP NNP NNP NNP NNP NNP VBZ PRP$ NN ) VBD DT JJ NN VBN IN NN NNS IN JJ NN CD TO VB DT NN IN JJ NNS VBG DT NN CC VBG DT JJ NN .)


In [ ]:
df_new_val=pd.DataFrame({'sentences':val_sentences,'NER_Tags':val_labels,'POS_Tags':val_POS_TAGS})   #,'Dependency_parsing':D_Tags})
df_new_val.head(5)

,sentences,NER_Tags,POS_Tags
0,for 14 - year - old amjad it is safer there th...,O B-Quantity I-Quantity I-Quantity I-Quantity ...,IN CD : NN : JJ NN PRP VBZ JJR RB IN VBG IN NN...
1,". near al huwayjah , one strike engaged an isi...",O O B-Location I-Location O B-Quantity B-Weapo...,". IN JJ NN , CD NN VBD DT JJ JJ NN CC VBD DT J..."
2,philip hammond condemns apparent barrel bomb a...,B-Person I-Person O O B-Weapon I-Weapon O O B-...,NN NN NN JJ NN NN NNS IN NN WDT VBP VBN CC VBN...
3,foreign secretary boris johnson welcomes the u...,B-Person I-Person I-Person I-Person O O B-Docu...,JJ NN VBD JJ VBZ DT JJ NN IN NN NNP VBD JJ IN ...
4,sadiq stood behind karim when the musician pla...,B-Person O O B-Person O B-Person I-Person O O ...,NN VBD IN NN WRB DT JJ VBD DT NN PRP '' NN VBD...


In [ ]:
# for i in range(len(test_sentences)):
#   temp=test_sentences[i].split()
#   pos=test_POS_TAGS[i].split()
#   # lab=val_labels[i].split()
#   if(len(temp)!= len(pos)):
#     print(f"For sentence number {i} : words: {len(temp)}  | pos: {len(pos)} ")# | NER: {len(lab)}")
#     # print(f" second last tag: {pos[-1]}")


In [ ]:
df_new_test=pd.DataFrame({'sentences':test_sentences,'POS_Tags':test_POS_TAGS})   #,'Dependency_parsing':D_Tags})
df_new_test.head(5)

,sentences,POS_Tags
0,carter thanked abadi for nearly two years of a...,"NN VBD NN IN RB CD NNS IN DT JJ JJ NN , CC VBD..."
1,analyses performed by the hospital lab include...,"NNS VBN IN DT NN NN VBP JJ , JJ , NN , JJ , CC..."
2,the last meeting of the small group took place...,"DT JJ NN IN DT JJ NN VBD NN IN NN , NN . NN . ..."
3,""" "" "" as we meet here , we are hoping to gener...","JJ NNP NNP IN PRP VBP RB , PRP VBP VBG TO VB D..."
4,one jet bombed a school for girls in a souther...,CD NN VBD DT NN IN NNS IN DT JJ NN IN DT NN IN...


In [ ]:
#len(POS_TAGS)== len(sentences)
#Creating a new dataframe with POS TAGS, Sentences and NER Tags
df_new_train=pd.DataFrame({'sentences':sentences,'NER_Tags':labels,'POS_Tags':POS_TAGS})   #,'Dependency_parsing':D_Tags})
df_new_train.head(5)

,sentences,NER_Tags,POS_Tags
0,operation steel curtain ( arabic : ا ل ح ج ا ب...,O O O O O O O O O O O O O O O O O O O O O O O ...,NN NN NN ( JJ : JJ NNP NNP NNP NNP NNP NNP NNP...
1,the hospital has facilities for mri and ct sca...,B-Location I-Location O O O O O O O O O O O O ...,"DT NN VBZ NNS IN NN CC JJ NN , DT NN NN CC JJ ..."
2,the operation was important in that it was the...,O O O O O O O O O O O O O O B-Organisation I-O...,DT NN VBD JJ IN DT PRP VBD DT JJ JJ JJ NN IN D...
3,this was my first visit to uzbekistan and an i...,O O B-Person O O O B-Location O O O O O O O O ...,DT VBD PRP$ JJ NN TO VB CC DT JJ NN TO VB CC V...
4,the group was founded by sheikh abu omar al - ...,B-Organisation I-Organisation O O O B-Person I...,DT NN VBD VBN IN JJ JJ NN SYM : NN . NN


## words, Tags to index

In [ ]:
#Referred to Lab 9 code
#word to index
word_to_ix = {}
for sentence in sentences+val_sentences+test_sentences:
    for word in sentence.split():
      word = word.lower()
      if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

#Tags to index
START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in labels+val_labels:
    for tag in tags.split():
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)

#POS Tags to index
pos_tag_to_ix={}
for tags in POS_TAGS+val_POS_TAGS+test_POS_TAGS:
    for tag in tags.split():
        if tag not in pos_tag_to_ix: # and tag not in ("''",'``','$',',','.',':'):
            pos_tag_to_ix[tag] = len(pos_tag_to_ix)

# #Dependency Tags to index
# dp_tag_to_ix={}
# for tags in D_Tags:
#     for tag in nltk.word_tokenize(tags):
#         if tag not in dp_tag_to_ix:
#             dp_tag_to_ix[tag] = len(dp_tag_to_ix)

### Glove Word Embedding

In [ ]:
#load pretrained model
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
#Embedding matrix
# Referred to Lab 9 code
EMBEDDING_DIM = 25

embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


array([[-0.54109001,  0.57410002, -0.57692999, ...,  0.28832   ,
        -0.95007998,  0.07306   ],
       [-0.70985001, -0.67839998,  0.1409    , ...,  1.13320005,
         0.30127001,  0.024852  ],
       [-1.22759998, -0.54803002,  1.18970001, ...,  1.63769996,
        -0.13262001, -0.45129001],
       ...,
       [-0.94571   ,  0.62142998, -0.28505999, ..., -0.014275  ,
        -0.17116   , -0.56494999],
       [ 0.070695  ,  0.97428   , -0.82848001, ..., -1.16050005,
        -0.38016999, -1.00199997],
       [-0.24102999,  0.40322   , -0.37015   , ..., -0.19942001,
        -0.74471998, -1.05780005]])

In [ ]:
# def prepare_sequence(seq, to_ix):
#     idxs = [to_ix[w] for w in seq]
#     return torch.tensor(idxs, dtype=torch.long)

In [ ]:
# sent_in=prepare_sequence("operation steel curtain".split(),word_to_ix)
# x(sent_in).view(len(sent_in),1,-1)

## Data indexing for model input

In [ ]:
 #Referred from Lab 9 code
#Convert data sets to indexes
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
      # if not flag:
      input_index_list.append([to_ix[w] for w in sent.split()])
      # else:
      #   temp=sent.split()
      #   input_index_list.append([to_ix[w] for w in temp])
    return input_index_list

#Training data
train_input_index =  to_index(df_new_train['sentences'],word_to_ix)
train_output_index = to_index(df_new_train['NER_Tags'],tag_to_ix)
train_pos_index = to_index(df_new_train['POS_Tags'],pos_tag_to_ix)

#Validation data
val_input_index = to_index(df_new_val['sentences'],word_to_ix)
val_output_index = to_index(df_new_val['NER_Tags'],tag_to_ix)
val_pos_index = to_index(df_new_val['POS_Tags'],pos_tag_to_ix)

#test data
test_input_index = to_index(df_new_test['sentences'],word_to_ix)
test_pos_index = to_index(df_new_test['POS_Tags'],pos_tag_to_ix)

# Bi-Lstm with CRF Module

In [ ]:
#Referred from Lab 9 code

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim,hidden_dim,pos_size=None,pos_tag_dim=None,no_layers=1,attention_type=None):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        self.nl=no_layers
        self.attention=attention_type

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))

        if pos_size is not None:
          self.pos_embeds=nn.Embedding(pos_size,pos_tag_dim)
          self.lstm = nn.LSTM(embedding_dim+pos_tag_dim, hidden_dim // 2,
                              num_layers=self.nl, bidirectional=True)
        else:
          self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                              num_layers=self.nl, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        if self.attention is None:
          self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        else:
          self.hidden2tag = nn.Linear(hidden_dim*2, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        # self.hidden =  nn.init.xavier_uniform(self.hidden2tag.weight)
        self.hidden=self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2 * self.nl, 1, self.hidden_dim // 2).to(device),
                torch.randn(2 * self.nl, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence,pos_tags=None):
      # try:
        embeds = []
        self.hidden = self.init_hidden()
        w_embeds = self.word_embeds(sentence)#.view(len(sentence), 1, -1)

        #check the embeding inputs and get vectors
        if pos_tags is not None:  #USes POS Tags and Word2Vec embedding
          p_embeds = self.pos_embeds(pos_tags)#.view(len(pos_tags),1,-1)
          embeds=torch.cat((w_embeds,p_embeds),1)
          embeds=embeds.view(len(embeds),1,-1)

        #only Word2Vec embedding
        else:
           embeds=w_embeds.view(len(w_embeds),1,-1)

        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)

        if self.attention is not None:
          lstm_out = self.calc_attention(lstm_out)

        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags,pos_tags=None):
        feats = self._get_lstm_features(sentence,pos_tags)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence,pos_tags=None):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence,pos_tags)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

    def calc_attention(self,h_states):

      if self.attention == "dot":
        score=h_states @ h_states.permute(1,0)
      elif self.attention == "scale":
        hidden_size = h_states.size(1)
        score=(1/(np.sqrt(hidden_size))*(h_states @ h_states.permute(1,0)))
      score=F.softmax(score,-1)
      res = score @ h_states
      return torch.cat([res,h_states],-1)



# Accuracy Function

In [ ]:
#function to calculate accuracy
#Referred from Lab 9 code
def cal_acc(model, input_index, output_index,pos_index=None):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        if pos_index is not None:
          score, pred = model(torch.tensor(idxs,dtype=torch.long).to(device),torch.tensor(pos_index[i],dtype=torch.long).to(device))
        else:
          score, pred = model(torch.tensor(idxs,dtype=torch.long).to(device).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    return predicted, ground_truth, accuracy

# Bi-LSTM with CRF and input embedding (glove + POS Tags)

In [ ]:
#hyperparameters
#Referred from Lab 9 code
START_TAG = "<START>"  #start of sequence
STOP_TAG = "<STOP>"    #end of seqeuence
EMBEDDING_DIM = 25     #embedding dimension
HIDDEN_DIM = 4          #hidden dimenisom
pos_tag_dim=3         #embedding dimnsion for POS TAG

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, len(pos_tag_to_ix), pos_tag_dim).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)


for epoch in range(20):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        pos_index=train_pos_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index,train_pos_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index,val_pos_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        pos_index=val_pos_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val loss: {val_loss:.2f}, val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 16368.24, train acc:0.7024 , val loss: 5191.54, val acc: 0.65,time:182.553055
Epoch: 2, Training loss: 10836.26, train acc:0.7296 , val loss: 4381.56, val acc: 0.67,time:182.917488
Epoch: 3, Training loss: 9240.30, train acc:0.7408 , val loss: 3978.03, val acc: 0.68,time:182.128889
Epoch: 4, Training loss: 8297.37, train acc:0.7656 , val loss: 3676.84, val acc: 0.71,time:179.73226
Epoch: 5, Training loss: 7628.15, train acc:0.7839 , val loss: 3486.84, val acc: 0.72,time:179.240387
Epoch: 6, Training loss: 7113.49, train acc:0.7898 , val loss: 3317.24, val acc: 0.73,time:178.94828
Epoch: 7, Training loss: 6709.05, train acc:0.7943 , val loss: 3206.83, val acc: 0.73,time:178.764359
Epoch: 8, Training loss: 6376.26, train acc:0.7990 , val loss: 3109.42, val acc: 0.74,time:179.008932
Epoch: 9, Training loss: 6098.95, train acc:0.8044 , val loss: 3020.90, val acc: 0.74,time:178.898362
Epoch: 10, Training loss: 5858.50, train acc:0.8082 , val loss: 2956.98, val acc: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

torch.save(model.state_dict(), '/content/drive/MyDrive/Kaggle/model.pt')

Mounted at /content/drive


In [ ]:
predicted=[]
for i,idxs in enumerate(test_input_index):
  score, pred = model(torch.tensor(idxs,dtype=torch.long).to(device),torch.tensor(test_pos_index[i],dtype=torch.long).to(device))
  predicted.append(pred)

In [ ]:
len(predicted)

199

In [ ]:
predicted=[]
for i,idxs in enumerate(test_input_index):
        score, pred = model(torch.tensor(idxs,dtype=torch.long).to(device),torch.tensor(test_pos_index[i],dtype=torch.long).to(device))
        predicted += pred

In [ ]:
df_temp=pd.DataFrame({'Predicted':predicted})

In [ ]:
# df_test_pred=pd.DataFrame({'sentences':test_sentences,'POS_Tags':test_POS_TAGS,'NER_Tags':predicted})   #,'Dependency_parsing':D_Tags})
# df_test_pred.head(5)

In [ ]:
predicted=[]
for i,idxs in enumerate(test_input_index):
  score, pred = model(torch.tensor(idxs,dtype=torch.long).to(device),torch.tensor(test_pos_index[i],dtype=torch.long).to(device))
  pred_labels=[]
  for preds in pred:
    for k,v in tag_to_ix.items():
      if v==preds:
        pred_labels.append(k)
  predicted.append(' '.join(pred_labels))

In [ ]:
df_temp.to_csv("/content/drive/MyDrive/Kaggle/predicted.csv",index=False)

# Bi-LSTM with CRF and input embedding (glove)

In [ ]:
#Referred from Lab 9 code
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix),tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        pos_index=val_pos_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val loss: {val_loss:.2f}, val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 16591.57, train acc:0.7067 , val loss: 5431.45, val acc: 0.65,time:180.24766
Epoch: 2, Training loss: 11580.35, train acc:0.7371 , val loss: 4574.84, val acc: 0.69,time:179.853611
Epoch: 3, Training loss: 9725.46, train acc:0.7594 , val loss: 4000.61, val acc: 0.71,time:179.499641
Epoch: 4, Training loss: 8496.96, train acc:0.7755 , val loss: 3660.18, val acc: 0.73,time:179.176019
Epoch: 5, Training loss: 7621.58, train acc:0.7860 , val loss: 3410.30, val acc: 0.73,time:180.74654
Epoch: 6, Training loss: 6944.95, train acc:0.7975 , val loss: 3220.17, val acc: 0.74,time:180.384671
Epoch: 7, Training loss: 6387.85, train acc:0.8082 , val loss: 3058.34, val acc: 0.75,time:180.911764
Epoch: 8, Training loss: 5908.08, train acc:0.8195 , val loss: 2923.17, val acc: 0.75,time:178.880586
Epoch: 9, Training loss: 5479.12, train acc:0.8279 , val loss: 2811.28, val acc: 0.76,time:179.313529
Epoch: 10, Training loss: 5126.69, train acc:0.8359 , val loss: 2728.98, val acc: 

# Stacked BiLSTM with CRF and input embedding (POS Tags + glove)

In [ ]:
#Referred from Lab 9 code
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4
pos_tag_dim=3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, len(pos_tag_to_ix), pos_tag_dim,2).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        pos_index=train_pos_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index,train_pos_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index,val_pos_index)

    # val_loss = 0
    # for i, idxs in enumerate(val_input_index):
    #     tags_index = val_output_index[i]
    #     pos_index=val_pos_index[i]
    #     sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
    #     targets = torch.tensor(tags_index, dtype=torch.long).to(device)
    #     pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)
    #     loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)
    #     val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 16566.47, train acc:0.7061 , val acc: 0.65,time:163.516722
Epoch: 2, Training loss: 10821.95, train acc:0.7187 , val acc: 0.66,time:163.501244
Epoch: 3, Training loss: 9367.29, train acc:0.7323 , val acc: 0.68,time:163.718022
Epoch: 4, Training loss: 8609.73, train acc:0.7271 , val acc: 0.68,time:166.244779
Epoch: 5, Training loss: 8105.54, train acc:0.7498 , val acc: 0.69,time:167.125295
Epoch: 6, Training loss: 7697.43, train acc:0.7434 , val acc: 0.69,time:167.048024
Epoch: 7, Training loss: 7394.33, train acc:0.7534 , val acc: 0.70,time:167.391112
Epoch: 8, Training loss: 7151.09, train acc:0.7597 , val acc: 0.70,time:166.877377
Epoch: 9, Training loss: 6846.31, train acc:0.7613 , val acc: 0.70,time:167.275137
Epoch: 10, Training loss: 6594.59, train acc:0.7664 , val acc: 0.71,time:166.934262


# Stacked BiLSTM with CRF and input embedding (glove)

In [ ]:
#Referred from Lab 9 code
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix),tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,no_layers=2).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)


for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f}, val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")

Epoch: 1, Training loss: 16516.75, train acc:0.7005, val acc: 0.65,time:166.395732
Epoch: 2, Training loss: 11947.01, train acc:0.7076, val acc: 0.66,time:166.956011
Epoch: 3, Training loss: 10147.21, train acc:0.7202, val acc: 0.66,time:166.749587
Epoch: 4, Training loss: 9112.02, train acc:0.7290, val acc: 0.67,time:166.729283
Epoch: 5, Training loss: 8449.77, train acc:0.7333, val acc: 0.68,time:167.066695
Epoch: 6, Training loss: 8016.83, train acc:0.7359, val acc: 0.68,time:166.687942
Epoch: 7, Training loss: 7632.13, train acc:0.7423, val acc: 0.69,time:166.95171
Epoch: 8, Training loss: 7268.78, train acc:0.7488, val acc: 0.69,time:166.787103
Epoch: 9, Training loss: 6936.15, train acc:0.7594, val acc: 0.71,time:167.573243
Epoch: 10, Training loss: 6587.86, train acc:0.7709, val acc: 0.72,time:167.834641


# Bi-LSTM with CRF and Attention

## Attention Type: DOT

### Stacked Bi-LSTM model with CRF,Attention and input embeddings (POS Tags+glove)

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4
pos_tag_dim=3
att_type='dot'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, len(pos_tag_to_ix), pos_tag_dim,no_layers=2,attention_type=att_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        pos_index=train_pos_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index,train_pos_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index,val_pos_index)

    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 17275.02, train acc:0.6999 , val acc: 0.65,time:268.153363
Epoch: 2, Training loss: 11655.80, train acc:0.7204 , val acc: 0.67,time:269.259289
Epoch: 3, Training loss: 9623.13, train acc:0.7275 , val acc: 0.67,time:264.292492
Epoch: 4, Training loss: 8673.70, train acc:0.7326 , val acc: 0.68,time:262.468029
Epoch: 5, Training loss: 8100.32, train acc:0.7441 , val acc: 0.70,time:261.089967
Epoch: 6, Training loss: 7640.14, train acc:0.7521 , val acc: 0.70,time:262.002614
Epoch: 7, Training loss: 7325.76, train acc:0.7584 , val acc: 0.70,time:263.529343
Epoch: 8, Training loss: 7057.94, train acc:0.7601 , val acc: 0.70,time:266.50174
Epoch: 9, Training loss: 6855.13, train acc:0.7635 , val acc: 0.71,time:259.686278
Epoch: 10, Training loss: 6618.14, train acc:0.7673 , val acc: 0.71,time:258.898479


### Bi-LSTM model with CRF,Attention and input embedding (POS Tags+glove)

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4
pos_tag_dim=3
att_type='dot'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, len(pos_tag_to_ix), pos_tag_dim,no_layers=1,attention_type=att_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        pos_index=train_pos_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index,train_pos_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index,val_pos_index)

    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 14651.37, train acc:0.7028 , val acc: 0.65,time:256.190524
Epoch: 2, Training loss: 10112.44, train acc:0.7362 , val acc: 0.68,time:258.07783
Epoch: 3, Training loss: 8548.32, train acc:0.7548 , val acc: 0.70,time:255.028611
Epoch: 4, Training loss: 7629.00, train acc:0.7649 , val acc: 0.71,time:252.840872
Epoch: 5, Training loss: 7034.17, train acc:0.7724 , val acc: 0.72,time:254.58089
Epoch: 6, Training loss: 6589.37, train acc:0.7861 , val acc: 0.74,time:253.188719
Epoch: 7, Training loss: 6231.56, train acc:0.8004 , val acc: 0.74,time:253.73631
Epoch: 8, Training loss: 5913.54, train acc:0.8061 , val acc: 0.75,time:259.929436
Epoch: 9, Training loss: 5609.84, train acc:0.8192 , val acc: 0.76,time:256.064089
Epoch: 10, Training loss: 5341.36, train acc:0.8272 , val acc: 0.77,time:258.234387


### Bi-LSTM Model with CRF,Attention and input embedding (glove)

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4
att_type='dot'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix),tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,attention_type=att_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 16375.66, train acc:0.6924 , val acc: 0.64,time:255.623344
Epoch: 2, Training loss: 10874.36, train acc:0.7172 , val acc: 0.66,time:256.409445
Epoch: 3, Training loss: 9286.71, train acc:0.7398 , val acc: 0.69,time:260.854605
Epoch: 4, Training loss: 8413.88, train acc:0.7574 , val acc: 0.70,time:261.739738
Epoch: 5, Training loss: 7753.25, train acc:0.7753 , val acc: 0.72,time:261.79014
Epoch: 6, Training loss: 7198.57, train acc:0.7866 , val acc: 0.73,time:260.924557
Epoch: 7, Training loss: 6734.81, train acc:0.7945 , val acc: 0.73,time:259.544075
Epoch: 8, Training loss: 6382.16, train acc:0.8014 , val acc: 0.74,time:257.833037
Epoch: 9, Training loss: 6057.28, train acc:0.8064 , val acc: 0.75,time:257.937455
Epoch: 10, Training loss: 5784.95, train acc:0.8107 , val acc: 0.75,time:257.204434


## Attention Type:Scale Dot

###  Bi-LSTM model with CRF,Attention and input embedding (POS Tags+glove)

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4
pos_tag_dim=3
att_type='scale'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, len(pos_tag_to_ix), pos_tag_dim,no_layers=1,attention_type=att_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        pos_index=train_pos_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index,train_pos_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index,val_pos_index)

    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 16357.44, train acc:0.7164 , val acc: 0.66,time:251.057102
Epoch: 2, Training loss: 10495.47, train acc:0.7482 , val acc: 0.69,time:255.417913
Epoch: 3, Training loss: 8697.47, train acc:0.7739 , val acc: 0.71,time:256.442811
Epoch: 4, Training loss: 7685.25, train acc:0.7878 , val acc: 0.72,time:257.441496
Epoch: 5, Training loss: 7008.07, train acc:0.7946 , val acc: 0.73,time:257.473708
Epoch: 6, Training loss: 6471.45, train acc:0.8040 , val acc: 0.74,time:256.986616
Epoch: 7, Training loss: 6031.34, train acc:0.8154 , val acc: 0.75,time:257.647965
Epoch: 8, Training loss: 5670.63, train acc:0.8234 , val acc: 0.76,time:255.127641
Epoch: 9, Training loss: 5393.67, train acc:0.8295 , val acc: 0.77,time:258.507399
Epoch: 10, Training loss: 5126.78, train acc:0.8346 , val acc: 0.77,time:257.833186


### Bi-LSTM Model with CRF,Attention and input embedding (glove)

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4
att_type='scale'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix),tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM,attention_type=att_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 16211.36, train acc:0.7151 , val acc: 0.66,time:162.413853
Epoch: 2, Training loss: 10794.68, train acc:0.7464 , val acc: 0.69,time:161.568904
Epoch: 3, Training loss: 8978.60, train acc:0.7651 , val acc: 0.71,time:161.400286
Epoch: 4, Training loss: 7959.25, train acc:0.7792 , val acc: 0.72,time:160.974011
Epoch: 5, Training loss: 7235.35, train acc:0.7938 , val acc: 0.73,time:160.653991
Epoch: 6, Training loss: 6600.03, train acc:0.8068 , val acc: 0.74,time:160.352876
Epoch: 7, Training loss: 6113.62, train acc:0.8185 , val acc: 0.74,time:160.530104
Epoch: 8, Training loss: 5685.13, train acc:0.8247 , val acc: 0.75,time:160.46274
Epoch: 9, Training loss: 5351.48, train acc:0.8329 , val acc: 0.75,time:160.241223
Epoch: 10, Training loss: 5066.29, train acc:0.8358 , val acc: 0.75,time:159.900378


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Kaggle/Stacked-BiLSTM-CRF-ATT-Glove.pt')

### Stacked Bi-LSTM model with CRF,Attention and input embeddings (POS Tags+glove)

In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 25
HIDDEN_DIM = 4
pos_tag_dim=3
att_type='scale'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# (self, vocab_size,pos_size, tag_to_ix, embedding_dim,pos_tag_dim, hidden_dim):
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, len(pos_tag_to_ix), pos_tag_dim,no_layers=2,attention_type=att_type).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(10):
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]
        pos_index=train_pos_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        pos_in_tags = torch.tensor(pos_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets,pos_in_tags)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index,train_pos_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index,val_pos_index)

    time2 = datetime.datetime.now()

    print(f"Epoch: {epoch+1}, Training loss: {train_loss:.2f}, train acc:{train_acc:.4f} , val acc: {val_acc:.2f},time:{(time2-time1).total_seconds()}")


Epoch: 1, Training loss: 17559.44, train acc:0.7028 , val acc: 0.65,time:163.945229
Epoch: 2, Training loss: 10801.32, train acc:0.7181 , val acc: 0.66,time:163.08931
Epoch: 3, Training loss: 9189.30, train acc:0.7290 , val acc: 0.68,time:162.707555
Epoch: 4, Training loss: 8388.87, train acc:0.7347 , val acc: 0.70,time:162.179478
Epoch: 5, Training loss: 7859.09, train acc:0.7371 , val acc: 0.69,time:161.861113
Epoch: 6, Training loss: 7487.66, train acc:0.7466 , val acc: 0.70,time:161.806712
Epoch: 7, Training loss: 7194.48, train acc:0.7511 , val acc: 0.70,time:162.036358
Epoch: 8, Training loss: 6927.87, train acc:0.7568 , val acc: 0.71,time:161.786919
Epoch: 9, Training loss: 6725.54, train acc:0.7600 , val acc: 0.71,time:161.510896
Epoch: 10, Training loss: 6475.68, train acc:0.7661 , val acc: 0.71,time:161.684695


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

torch.save(model.state_dict(), '/content/drive/MyDrive/Kaggle/Stacked-BiLSTM-CRF-ATT-POS_Gl.pt')

Mounted at /content/drive
